In [ ]:
import scanpy as sc
import numpy as np
from matplotlib.patches import Rectangle
import seaborn as sns 
import matplotlib.pyplot as plt
import os 
from scipy.stats import mannwhitneyu

## Load data and set up params

In [ ]:
adata = sc.read_h5ad('/stanley/WangLab/kamal/data/mouse/gut/ileum_spin_filtered.h5ad')
basepath = '/stanley/WangLab/kamal/code/remote_notebooks/gut_xcondition/figures_kamal/'
dpi = 300
ctypes = adata.obs['general_annos'].unique()
ctype_cmap = {ctypes[i]:sc.pl.palettes.default_102[i] for i in range(len(ctypes))}
region_cmap = {str(i):sc.pl.palettes.default_102[i] for i in range(100)}
figsize = (7,4.5)

In [ ]:
adata

In [ ]:
# Get vmin and vmax of gene expression across samples for colorbar normalization
def getMinMax(adata, gene, layer='smooth'):
    return (adata[:,gene].layers[layer].min(), adata[:,gene].layers[layer].max())

# Full tissue slices

## Tissue and latent colored by cell type

In [ ]:
sc.set_figure_params(figsize=figsize)
sc.pl.embedding(adata, basis='spatial', color='general_annos', palette=ctype_cmap, s=3, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'tissue_colored_by_celltype.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap', color='general_annos', palette=ctype_cmap, s=3, title='', legend_loc='', return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_celltype.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap', color='sample', s=3, title='', legend_loc='', return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_condition_celltype.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
random_idxs = np.random.choice(np.arange(len(adata)), size=len(adata), replace=False)
sc.pl.embedding(adata[random_idxs], basis='X_umap', color='sample', s=3, title='', legend_loc='', return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_condition_celltype.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(10,10))
sc.pl.embedding(adata, basis='X_umap', color='general_annos', palette=ctype_cmap, s=10, title='', legend_loc='on data')

In [ ]:
sc.set_figure_params(figsize=(10,10))
sc.pl.embedding(adata, basis='X_umap', color='sample', s=10, title='')

## Tissue and latent colored by region

In [ ]:
sc.set_figure_params(figsize=figsize)
sc.pl.embedding(adata, basis='spatial', color='region', palette=region_cmap, s=3, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'tissue_colored_by_region.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_spin', color='region', palette=region_cmap, s=3, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_region.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_spin', color='sample', s=3, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_condition.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
random_idxs = np.random.choice(np.arange(len(adata)), size=len(adata), replace=False)
sc.pl.embedding(adata[random_idxs], basis='X_umap_spin', color='sample', s=3, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_condition.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

# Zooms

## Get zoom regions

In [ ]:
def get_zoom(adata, x, y, width, height, theta):
    x1, x2 = x, x+width
    y1, y2 = y, y+height
    R = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])
    rdata = adata.copy()
    rdata.obsm['spatial'] = rdata.obsm['spatial'] @ R.T
    zoomdata = rdata[rdata.obsm['spatial'][:,0]>x1]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,0]<x2]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]>y1]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]<y2]
    return zoomdata

In [ ]:
wti_adata = adata[adata.obs['sample']=='wt']

theta = -0.2
R = np.array([
    [np.cos(theta), -np.sin(theta)],
    [np.sin(theta), np.cos(theta)]
])
rdata = wti_adata.copy()
rdata.obsm['spatial'] = rdata.obsm['spatial'] @ R.T
wti_box = (6800, 10200, 3000, 1000)
c = [region_cmap[region] for region in rdata.obs['region']]
plt.figure(figsize=(5,5))
plt.scatter(*rdata.obsm['spatial'].T, s=1, c=c)
rect = Rectangle(wti_box[:2], wti_box[2], wti_box[3], fill=False, edgecolor='k')
plt.gca().add_patch(rect)
plt.axis('off')
plt.show()

In [ ]:
gfi_adata = adata[adata.obs['sample']=='gf']

theta = 0.35
R = np.array([
    [np.cos(theta), -np.sin(theta)],
    [np.sin(theta), np.cos(theta)]
])
rdata = gfi_adata.copy()
rdata.obsm['spatial'] = rdata.obsm['spatial'] @ R.T
# gfi_box = (27400, 28500, 3000, 1000)
gfi_box = (27400, 27600, 3000, 1000)
c = [region_cmap[region] for region in rdata.obs['region']]
plt.figure(figsize=(8,10))
plt.scatter(*rdata.obsm['spatial'].T, s=1, c=c)
rect = Rectangle(gfi_box[:2], gfi_box[2], gfi_box[3], fill=False, edgecolor='k')
plt.gca().add_patch(rect)
plt.axis('off')
plt.show()

In [ ]:
wti_zoom = get_zoom(wti_adata, *wti_box, -0.2)
wti_zoom.obsm['spatial'] = wti_zoom.obsm['spatial'][:,::-1]
gfi_zoom = get_zoom(gfi_adata, *gfi_box, 0.35)
gfi_zoom.obsm['spatial'][:,[0,1]] = gfi_zoom.obsm['spatial'][:,[1,0]]
zoomdatas = [wti_zoom, gfi_zoom]
conditions = ['WT', 'GF']

In [ ]:
s = 200

## Tissue zooms colored by region

In [ ]:
# Color by region
sc.set_figure_params(figsize=(2,6))
for i in range(len(conditions)):
    figname = f'zoom_tissue_{conditions[i]}_colored_by_region'
    # savepath = os.path.join(basepath, figname)
    # viz.plotHulls(zoomdatas[i], hulls[i], cells[i], color_by='region', cmap=region_cmap, scale=800, scale_bar=True, save_path=savepath)
    # plt.show()
    # plt.close()
    # for region in np.unique(zoomdatas[i].obs['region']):
    # for region in ['0']: # only plot the relevant region
    # figname = f'zoom_tissue_{conditions[i]}_colored_by_region_{region}'
    savepath = os.path.join(basepath, figname)
    # viz.plotHulls(zoomdatas[i], hulls[i], cells[i], color_by='region', group=region, cmap=region_cmap, scale=800, save_path=savepath)
    # plt.show()
    # plt.close()
    sc.pl.embedding(zoomdatas[i], basis='spatial', color='region', palette=region_cmap, s=s, title='', legend_loc=None, return_fig=True, edgecolor='k', linewidth=0.05)
    plt.axis('off')
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
# Color by region
sc.set_figure_params(figsize=(2,6))
for i in range(len(conditions)):
    figname = f'zoom_tissue_{conditions[i]}_colored_by_celltype'
    # savepath = os.path.join(basepath, figname)
    # viz.plotHulls(zoomdatas[i], hulls[i], cells[i], color_by='region', cmap=region_cmap, scale=800, scale_bar=True, save_path=savepath)
    # plt.show()
    # plt.close()
    # for region in np.unique(zoomdatas[i].obs['region']):
    # for region in ['0']: # only plot the relevant region
    # figname = f'zoom_tissue_{conditions[i]}_colored_by_region_{region}'
    savepath = os.path.join(basepath, figname)
    # viz.plotHulls(zoomdatas[i], hulls[i], cells[i], color_by='region', group=region, cmap=region_cmap, scale=800, save_path=savepath)
    # plt.show()
    # plt.close()
    sc.pl.embedding(zoomdatas[i], basis='spatial', color='general_annos', palette=ctype_cmap, s=s, title='', legend_loc=None, return_fig=True, edgecolor='k', linewidth=0.05)
    plt.axis('off')
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

# Shared and condition-specific region markers

In [ ]:
# Shared
region = '1'
sc.tl.rank_genes_groups(adata, groupby='region')#, layer='smooth')
marker_df = sc.get.rank_genes_groups_df(adata, group=region)
shared_marker = marker_df['names'][0]

In [ ]:
plt.figure(figsize=(5,2))
plt.hist(marker_df['scores'], bins=30, color=region_cmap[region], linewidth=1, edgecolor='k')
plt.grid(None)
plt.axis('off')
figname = f'region_deg_score_hist.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
plt.figure(figsize=(5,2))
plt.hist(marker_df['scores'], bins=30, color=region_cmap[region], linewidth=1, edgecolor='k')
plt.grid(None)
# plt.axis('off')
# figname = f'region_deg_score_hist.png'
# savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
marker_df['scores']

In [ ]:
marker_df['names']

In [ ]:
# Unique
xdata = adata[adata.obs['region']==region]
sc.tl.rank_genes_groups(xdata, groupby='sample')#, layer='smooth')
marker_df = sc.get.rank_genes_groups_df(xdata, group='wt')
wt_markers = marker_df['names']
wt_marker = wt_markers[0]

In [ ]:
plt.figure(figsize=(5,2))
plt.hist(marker_df['scores'], bins=30, color=region_cmap[region], linewidth=1, edgecolor='k')
plt.grid(None)
plt.axis('off')
figname = f'condition_vil_deg_score_hist.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
plt.figure(figsize=(5,2))
plt.hist(marker_df['scores'], bins=30, color=region_cmap[region], linewidth=1, edgecolor='k')
plt.grid(None)
# plt.axis('off')
# figname = f'region_deg_score_hist.png'
# savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
marker_df['scores']

In [ ]:
marker_df['names']

In [ ]:
X = sc.concat((zoomdatas[0], zoomdatas[1]))[:,[shared_marker,wt_marker]].X
vmax = X.max()
vmin = X.min()
cmaps = ['Blues', 'Oranges']

In [ ]:
print(shared_marker)
sc.set_figure_params(figsize=(2,6))
vmin, vmax = getMinMax(adata, shared_marker)

# Plot region 1 shared marker for WT and GF
for i in range(len(conditions)):
    figname = f'zoom_tissue_{conditions[i]}_colored_by_{shared_marker}.png'
    savepath = os.path.join(basepath, figname)
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=shared_marker, cmap=cmaps[i], s=220,
    title='', colorbar_loc=None, return_fig=True, edgecolor='k', linewidth=0.02)
    plt.axis('off')
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
print(wt_marker)
sc.set_figure_params(figsize=(2,6))
vmin, vmax = getMinMax(adata, wt_marker)

# Plot region 1 shared marker for WT and GF
for i in range(len(conditions)):
    figname = f'zoom_tissue_{conditions[i]}_colored_by_{wt_marker}.png'
    savepath = os.path.join(basepath, figname)
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=wt_marker, cmap=cmaps[i], s=220,
    title='', colorbar_loc=None, return_fig=True, edgecolor='k', linewidth=0.05)
    plt.axis('off')
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

# Tissue zooms and latent colored by DiffMap

In [ ]:
# omit_regions = ['8', '2', '3', '4'] # villi only for demo
# omit_regions = ['8']
# keep_regions = [region for region in adata.obs['region'].unique() if region not in omit_regions]
# adata_all = adata.copy()
# adata = adata[np.isin(adata.obs['region'], keep_regions)]

In [ ]:
# redo DiffMap without adipose
sc.tl.diffmap(adata, neighbors_key='region')
adata.obs['depth'] = adata.obsm['X_diffmap'][:,1]

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_spin', color='depth', s=3, title='', colorbar_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_depth.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
# vmax = adata.obs['depth'].max()
# vmin = adata.obs['depth'].min()

# sc.set_figure_params(figsize=(6,6))
# plt.scatter(*adata_all.obsm['X_umap_spin'].T, color=[0.7]*3, s=0.2)
# plt.scatter(*adata.obsm['X_umap_spin'].T, c=adata.obs['depth'], cmap='viridis_r', vmin=vmin, vmax=vmax, s=0.2)
# plt.axis('off')
# figname = f'latent_colored_by_depth.png'
# savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
# plt.show()

In [ ]:
wti_adata = adata[adata.obs['sample']=='wt']
gfi_adata = adata[adata.obs['sample']=='gf']
wti_zoom = get_zoom(wti_adata, *wti_box, -0.2)
wti_zoom.obsm['spatial'] = wti_zoom.obsm['spatial'][:,::-1]
gfi_zoom = get_zoom(gfi_adata, *gfi_box, 0.35)
gfi_zoom.obsm['spatial'][:,[0,1]] = gfi_zoom.obsm['spatial'][:,[1,0]]
zoomdatas = [wti_zoom, gfi_zoom]
conditions = ['wt', 'gf']

In [ ]:
vmax = adata.obs['depth'].max()
vmin = adata.obs['depth'].min()

# Plot region 1 shared marker for WT and GF
for i in range(len(conditions)):
    figname = f'zoom_tissue_{conditions[i]}_colored_by_depth.png'
    savepath = os.path.join(basepath, figname)
    sc.set_figure_params(figsize=(2,6))
    sc.pl.embedding(zoomdatas[i], basis='spatial', color='depth', cmap='viridis', s=s, title='', colorbar_loc=None, return_fig=True)
    plt.axis('off')
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
# colorbar only
fig = plt.figure()
cax = plt.imshow(np.array([[0,1]]), cmap='viridis')
plt.gca().set_visible(False)
cbar = fig.colorbar(cax, ticks=[0, 1])
cbar.ax.set_yticklabels(['', ''])
figname = 'cbar_color_by_expr.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath)
plt.show()

In [ ]:
# colorbar only
fig = plt.figure()
cax = plt.imshow(np.array([[0,1]]), cmap='Blues')
plt.gca().set_visible(False)
cbar = fig.colorbar(cax, ticks=[0, 1])
cbar.ax.set_yticklabels(['', ''])
figname = 'cbar_Blues.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath)
plt.show()

In [ ]:
# colorbar only
fig = plt.figure()
cax = plt.imshow(np.array([[0,1]]), cmap='Oranges')
plt.gca().set_visible(False)
cbar = fig.colorbar(cax, ticks=[0, 1])
cbar.ax.set_yticklabels(['', ''])
figname = 'cbar_Oranges.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath)
plt.show()

# Continuous depth vs gene

In [ ]:
adata.obs['depth'] *= -1

In [ ]:
# Make bins
num_bins = 50
max_depth = adata.obs['depth'].max()
min_depth = adata.obs['depth'].min()
increments = (max_depth-min_depth)/num_bins
bins = np.arange(min_depth, max_depth, increments)
adata.obs['depth_bin'] = np.digitize(adata.obs['depth'], bins=bins) - 1 # account for idxs

# Make expression nonnegative
adata.X -= adata.X.min()
# adata.layers['smooth'] -= adata.layers['smooth'].min()

# Separate species
adata_wt = adata[adata.obs['sample']=='wt']
adata_gf = adata[adata.obs['sample']=='gf']
adatas = [adata_wt, adata_gf]

In [ ]:
def get_hists(adatas, gene, start_bin=None, end_bin=None):

    hists = []
    for subdata in adatas:

        # Initialize a few vars
        hist = np.zeros(num_bins)
        depth = subdata.obs['depth_bin'].values
        expr = subdata[:,gene].X.flatten()
        # expr = subdata[:,gene].layers['smooth'].flatten()

        # Wanted to just use np.histogram here to get counts, but it has known binning diffs from np.digitize
        bin_counts = np.zeros(num_bins)
        nonzero_counts = subdata.obs['depth_bin'].value_counts().sort_index()
        for i in nonzero_counts.index:
            bin_counts[i] = nonzero_counts.loc[i]

        # Add each cell's expression into corresponding cell depth bin
        for i in range(len(subdata)):
            hist[depth[i]] += expr[i] / bin_counts[depth[i]]
        
        # Omit highest and lowest depths due to noise
        if not start_bin and end_bin:
            start_bin = num_bins // 10 # omit bottom 10% of depth
            end_bin = num_bins // 10 * 9 # omit top 10% of depth
        hist = hist[start_bin:end_bin]

        # Normalize wrt given dataset
        hist /= hist.sum()
        hists.append(hist)
    
    return hists

In [ ]:
start_bin = 15
end_bin = 45

In [ ]:
for gene in ['Reg3g', 'Slc10a2', 'Vil1', 'Ano1']:
    print(gene)
    hists = get_hists(adatas, gene, start_bin=start_bin, end_bin=end_bin)
    plt.figure(figsize=(10,3))
    for i in range(len(hists)):
        plt.plot(hists[i], label=conditions[i], linewidth=5)
    plt.axis('off')
    figname = f'depth_curve_{gene}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
from tqdm import tqdm
n_genes = len(adata.var_names)
wt_hists = np.zeros((n_genes, end_bin-start_bin))
gf_hists = np.zeros((n_genes, end_bin-start_bin))
for i in tqdm(range(n_genes)):
    hists = get_hists(adatas, adata.var_names[i], start_bin=start_bin, end_bin=end_bin)
    wt_hists[i] = hists[0]
    gf_hists[i] = hists[1]

In [ ]:
for gene in ['Reg3g', 'Slc10a2', 'Vil1', 'Ano1']:
    print(gene)
    plt.figure(figsize=(10,3))
    plt.plot(adata[:,gene].varm['wt_depth'].flatten(), linewidth=5)
    plt.plot(adata[:,gene].varm['gf_depth'].flatten(), linewidth=5)
    plt.axis('off')

In [ ]:
adata.varm['wt_depth'] = wt_hists
adata.varm['gf_depth'] = gf_hists

In [ ]:
# adata.write('/stanley/WangLab/kamal/code/remote_notebooks/gut_xcondition/adata_depth_processed.h5ad')
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/gut_xcondition/adata_depth_processed.h5ad')

In [ ]:
# from tqdm import tqdm
# hist_dict = dict()
# for gene in tqdm(adata.var_names):
#     hist_dict[gene] = get_hists(adatas, gene, start_bin=start_bin, end_bin=end_bin)

In [ ]:
adata.var['cond_expr_diffs'] = np.absolute(adata.varm['wt_depth'] - adata.varm['gf_depth']).sum(axis=1)

In [ ]:
num_top_genes = 5
most_similar_genes = adata.var['cond_expr_diffs'].sort_values()[:num_top_genes].index
most_different_genes = adata.var['cond_expr_diffs'].sort_values()[-num_top_genes:][::-1].index

In [ ]:
for gene in most_similar_genes[:5]:
    print(gene)
    plt.figure(figsize=(10,3))
    plt.plot(adata[:,gene].varm['wt_depth'].flatten(), linewidth=5)
    plt.plot(adata[:,gene].varm['gf_depth'].flatten(), linewidth=5)
    plt.axis('off')
    figname = f'depth_curve_{gene}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
for gene in most_different_genes[:5]:
    print(gene)
    plt.figure(figsize=(10,3))
    plt.plot(adata[:,gene].varm['wt_depth'].flatten(), linewidth=5)
    plt.plot(adata[:,gene].varm['gf_depth'].flatten(), linewidth=5)
    plt.axis('off')
    figname = f'depth_curve_{gene}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
plt.figure(figsize=(6,2))
plt.hist(adata.var['cond_expr_diffs'], bins=30, color='k')
plt.xlabel('Condition Difference')
plt.ylabel('Count')
plt.axis('off')
figname = f'gene_depth_diffs_hist.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
plt.figure(figsize=(6,2))
plt.hist(adata.var['cond_expr_diffs'], bins=30, color='k')
plt.xlabel('Condition Difference')
plt.ylabel('Count')
plt.show()

In [ ]:
for gene in ['Reg3b', 'Slc10a2', 'Vil1']:
    print(adata[:,gene].var['cond_expr_diffs'])

In [ ]:
# adata.var['cond_expr_diffs'].loc['Vil1']

In [ ]:
# # Find genes that are closer to the surface in WT than GF
# from scipy.stats import wilcoxon

# ps = np.zeros(n_genes)

# for i in tqdm(range(n_genes)):

#     wt = adata.varm['wt_depth'][i]
#     gf = adata.varm['gf_depth'][i]
#     # ps[i] = wilcoxon(wt, gf, alternative='greater')[1]
#     ps[i] = wilcoxon(wt, gf, alternative='two-sided')[1]

# adata.var['depth_diff'] = ps

In [ ]:
# adata.var['depth_diff'].sort_values(ascending=True)

In [ ]:
cmaps = ['Blues', 'Oranges']

In [ ]:
gene = 'Slc8a1'
sc.set_figure_params(figsize=(2,6))
vmin, vmax = getMinMax(adata, gene)

# Plot region 1 shared marker for WT and GF
for i in range(len(conditions)):
    figname = f'zoom_tissue_{conditions[i]}_colored_by_{gene}.png'
    savepath = os.path.join(basepath, figname)
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=gene, cmap=cmaps[i], s=220,
    title='', colorbar_loc=None, return_fig=True, edgecolor='k', linewidth=0.02)
    plt.axis('off')
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
gene = 'Ano1'
sc.set_figure_params(figsize=(2,6))
vmin, vmax = getMinMax(adata, gene)

# Plot region 1 shared marker for WT and GF
for i in range(len(conditions)):
    figname = f'zoom_tissue_{conditions[i]}_colored_by_{gene}.png'
    savepath = os.path.join(basepath, figname)
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=gene, cmap=cmaps[i], s=220,
    title='', colorbar_loc=None, return_fig=True, edgecolor='k', linewidth=0.02)
    plt.axis('off')
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
gene = 'Slc10a2'
sc.set_figure_params(figsize=(2,6))
vmin, vmax = getMinMax(adata, gene)

# Plot region 1 shared marker for WT and GF
for i in range(len(conditions)):
    figname = f'zoom_tissue_{conditions[i]}_colored_by_{gene}.png'
    savepath = os.path.join(basepath, figname)
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=gene, cmap=cmaps[i], s=220,
    title='', colorbar_loc=None, return_fig=True, edgecolor='k', linewidth=0.02)
    plt.axis('off')
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
adata.write('/stanley/WangLab/kamal/code/remote_notebooks/gut_xcondition/adata_depth_processed.h5ad')

# Cell type vs depth

In [ ]:
# adata = sc.read_h5ad('/stanley/WangLab/kamal/data/mouse/gut/ileum_filtered_genes_spin_50_16_depth.h5ad')
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/gut_xcondition/adata_depth_processed.h5ad')

In [ ]:
wti_adata = adata[adata.obs['sample']=='wt']
gfi_adata = adata[adata.obs['sample']=='gf']

In [ ]:
# Set up variables
ctype_key = 'general_annos'
ctypes = adata.obs[ctype_key].unique()
ns = dict()
alphas = dict()
significances = dict()

# Set up p-values
reject_p = 0.01
corrected_reject_p = reject_p / len(ctypes) # Bonferroni correction

# Get statistics
for ctype in ctypes:
    wt = wti_adata[wti_adata.obs[ctype_key]==ctype].obs['depth']
    gf = gfi_adata[gfi_adata.obs[ctype_key]==ctype].obs['depth']
    alpha = mannwhitneyu(wt, gf).pvalue
    ns[ctype] = (len(wt), len(gf))
    alphas[ctype] = alpha
    significances[ctype] = alpha < corrected_reject_p

In [ ]:
conditions_cmap = plt.cm.get_cmap('tab10').colors[:2][::-1]

In [ ]:
print(list(adata.obs['general_annos'].unique()))

In [ ]:
order = list(adata.obs['general_annos'].unique())
order.remove('Immune')
order.append('Immune')

In [ ]:
order

In [ ]:
# Make main plot
plt.figure(figsize=(12,1.7))
sns.boxplot(
    data=adata.obs,
    x=ctype_key,
    y='depth',
    hue='sample',
    palette=conditions_cmap[::-1],
    order=order
)
plt.gca().grid(False)
# plt.legend(bbox_to_anchor=(1,1))
plt.gca().get_legend().remove()

# Add significance indicators
max_depth = adata.obs['depth'].max()
buffer = 0.001
sns.despine()
ticks, labels = plt.xticks()
for i in range(len(ticks)):
    label = labels[i].get_text().split(',')[0]
    significant = significances[label]
    if significant:
        tick = ticks[i]
        plt.text(tick, max_depth+buffer, '*', ha='center')
        plt.plot([tick-0.3, tick+0.3], [max_depth+buffer, max_depth+buffer], color='k')

# Modify xticks
new_labels = []
for i in range(len(ticks)):
    label = labels[i].get_text().split(',')[0]
    # new_labels.append(f'{label}, n={ns[label]}')
    new_labels.append('')
plt.xticks(ticks, new_labels, rotation=90)

# Modify yticks
vmax = adata.obs['depth'].max()
vmin = adata.obs['depth'].min()
plt.yticks([vmax, vmin], ['', ''])
plt.gca().set_yticklabels('')
plt.xlabel('')
plt.ylabel('')

# plt.axis('off')
# plt.title('Ileum', fontsize=18)
figname = 'ctype_depth_boxplot.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
ctype = 'Immune'
sc.set_figure_params(figsize=(2,6))

for i in range(len(conditions)):
    figname = f'zoom_tissue_{conditions[i]}_colored_by_{ctype}'
    savepath = os.path.join(basepath, figname)
    sc.pl.embedding(zoomdatas[i], basis='spatial', color='general_annos', groups=[ctype], palette=[conditions_cmap[::-1][i]], s=s, title='', legend_loc=None, return_fig=True)
    plt.axis('off')
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

## Plot hist for physical depth in each slice

In [ ]:
# Isolate a given cell type
ctype_key = 'general_annos'
# ctype = 'Lymphocyte'
# ctype = 'Fibroblast'
zoomdata_wt_ctype = wti_zoom[wti_zoom.obs[ctype_key]==ctype]
zoomdata_gf_ctype = gfi_zoom[gfi_zoom.obs[ctype_key]==ctype]

In [ ]:
physdepth_wt = zoomdata_wt_ctype.obsm['spatial'][:,1].copy().astype(float)
physdepth_wt -= physdepth_wt.min()
physdepth_wt /= physdepth_wt.ptp()
plt.figure(figsize=(10,1))
plt.hist(physdepth_wt, color=conditions_cmap[1])
plt.axis('off')
figname = f'hist_for_zoom_wt_{ctype}.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
physdepth_gf = zoomdata_gf_ctype.obsm['spatial'][:,1].copy().astype(float)
physdepth_gf -= physdepth_gf.min()
physdepth_gf /= physdepth_gf.ptp()
plt.figure(figsize=(10,1))
plt.hist(physdepth_gf, color=conditions_cmap[0])
plt.axis('off')
figname = f'hist_for_zoom_gf_{ctype}.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()